In [116]:
#!/usr/bin/env python3
"""
Read SO-CI output from Molpro.
Assign omegas, including parity. 
Also show composition of a level, or distribution of a term.
C2v symmetry (4 irreps) is assumed. 
KKI March 2023: include dipole moment
"""
import re, sys, copy, glob, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, '../atomic_SOC')
import molpro_subs as mpr
import chem_subs as chem

pd.set_option('display.width', 1000)

In [382]:
fsoci = 'ac5z_hybB_r2p2444_lz.pro'
fsoci = '../UMemphis/mgcl-equil-karl-block.pro'
#fsoci = '../UMemphis/mgcl_small4.pro'
print('Will read SO-CI info from file: {:s}'.format(fsoci))

Will read SO-CI info from file: ../UMemphis/mgcl-equil-karl-block.pro


In [383]:
# Read CASSCF
PG = mpr.read_point_group(fsoci)
print('Point group is', PG)
crd, lineno_crd = mpr.read_coordinates(fsoci, linenum=True)
if isinstance(lineno_crd, list):
    # take last geometry
    crd = crd[-1]
    lineno_crd = lineno_crd[-1]
# get diatomic bond length
G = chem.Geometry(crd, intype='DataFrame', units='bohr')
G.toAngstrom()
R = np.round(G.distance(0, 1), 6)  # round the bond length to 6 digits
print('Bond length = {:.4f}'.format(R))
caslist, lineno_cas = mpr.readMULTI(fsoci, PG=PG, linenum=True)
CAS = caslist[-1]   # assume the last CASSCF to be the relevant one

oldcas = CAS.results.copy()
for Spin in sorted(set(oldcas.Spin)):
    nspin = len(oldcas[oldcas.Spin == Spin])
    print('{:d} {:s}s'.format(nspin, Spin))
print('Active space = {:d}/{:d}'.format(CAS.nactel(), CAS.nactorb()))
CAS.results = mpr.relabel_CAS_by_energy(CAS.results)
# Note any changes in CAS labeling
diflbl = (oldcas.Label != CAS.results.Label).values
if diflbl.any():
    dflbl = oldcas[['Label', 'Term']].copy()
    dflbl['NewLabel'] = CAS.results.Label
    print('Some CAS labels changed:')
    display(dflbl)
# check for dynamical weights
rx_dynw = re.compile('[~!]*dynw,(\d+)')
dynw = 0
with open(fsoci, 'r') as F:
    for line in F:
        m = rx_dynw.search(line)
        if m:
            dynw = int(m.group(1))
            print('Dynamical weighting with dynw = {:d}'.format(dynw))
if not dynw:
    print('Uniform weighting')

Point group is C2v
Bond length = 2.3000
22 Doublets
Active space = 9/8
Uniform weighting


In [384]:
# count the terms that are included in the calculation
from collections import Counter
for spin, grp in CAS.results.groupby('Spin'):
    print(spin)
    print(Counter(grp.Term.tolist()))
#CAS.results

Doublet
Counter({'2Π': 10, '2Σ+': 6, '2Δ': 4, '2Σ-': 2})


In [385]:
casdf = CAS.results[['Irrep', 'Label', 'Energy', 'Term']].copy()
casdf['S'] = [chem.spinname(x)-1 for x in CAS.results.Spin]
casdf['Lz'] = np.round(np.sqrt(np.abs(CAS.results.LzLz)), 0).astype(int)
spin = sorted(set(casdf.S))
irreps = sorted(set(casdf.Irrep))
lzvals = sorted(set(casdf.Lz))

In [386]:
casdf['ecm'] = np.round((casdf.Energy - casdf.Energy.min()) * chem.AU2CM, 0)
casdf.sort_values('Energy')

,Irrep,Label,Energy,Term,S,Lz,ecm
0,1,1.1,-660.906685,2Σ+,1,0,0.0
8,2,1.2,-660.787376,2Π,1,1,26185.0
13,3,1.3,-660.787376,2Π,1,1,26185.0
9,2,2.2,-660.775209,2Π,1,1,28855.0
14,3,2.3,-660.775209,2Π,1,1,28855.0
1,1,2.1,-660.748771,2Σ+,1,0,34658.0
2,1,3.1,-660.730059,2Σ+,1,0,38765.0
3,1,4.1,-660.718892,2Δ,1,2,41216.0
18,4,1.4,-660.718892,2Δ,1,2,41216.0
19,4,2.4,-660.716449,2Σ-,1,0,41752.0


In [387]:
# check for proper pairing of degenerate states
broken = False
for grp, dfg in casdf.groupby(['S', 'Lz']):
    if grp[1] == 0:
        # ignore Sigma states
        continue
    # number in each irrep should be equal
    grpi = dfg.groupby('Irrep')
    lens = grpi.size().values
    if lens[0] != lens[1]:
        print('Broken pair somewhere')
        display(dfg.sort_values('Energy'))
        broken = True
if not broken:
    print('All CASSCF state pairs are closed')

All CASSCF state pairs are closed


In [388]:
# check for non-integer Lz values
cruft = casdf.Lz - np.round(casdf.Lz, 0)
if cruft.any():
    print('*** There are non-integer values of Lz')
else:
    print('Lz values look OK')

Lz values look OK


In [389]:
# order the CASSCF states by energy within each (S, irrep) group
newdf = pd.DataFrame(columns=casdf.columns)
for lbl, dg in casdf.groupby(['S', 'Irrep']):
    dt = dg.copy().sort_values('Energy').reset_index(drop=True)
    #display(dt)
    newdf = newdf.append(dt)

In [390]:
# read MRCI
cilist, lineno_ci = mpr.readMRCI(fsoci, linenum=True)   # probably many
for m in cilist:
    m.transfer_lz(CAS.results)
mrci = [mpr.MRCIstate(row) for m in cilist for (irow, row) in m.results.iterrows()]
dfci = mpr.combineMRCI(cilist)

In [391]:
dfci['ecm'] = np.round( (dfci.Edav - dfci.Edav.min()) * chem.AU2CM, 1)
dfci.sort_values('Edav')

,Group,Spin,Irrep,Label,Energy,Edav,Ncore,dipX,dipY,dipZ,Eref,Dipole,Ref,C0,Configs,Lz,Term,ecm
0,1,Doublet,1,1.1,-661.161910,-661.189203,10,0.0,0.0,-1.564913,-660.906685,1.564913,1.1,0.950992,"{'22a02020': 0.9269158, '220a2020': -0.1195967...",0.0,2Σ+,0.0
8,3,Doublet,2,1.2,-661.037247,-661.069028,10,0.0,0.0,-1.980664,-660.787376,1.980664,1.2,0.942952,"{'22aab020': -0.0199338, '22aba020': 0.0384188...",1.0,2Π,26375.4
13,4,Doublet,3,1.3,-661.037247,-661.069028,10,0.0,0.0,-1.980664,-660.787376,1.980664,1.3,0.942952,"{'22aa20b0': -0.0199338, '22ab20a0': 0.0384188...",1.0,2Π,26375.4
14,4,Doublet,3,2.3,-661.003948,-661.025732,10,0.0,0.0,1.420276,-660.775209,1.420276,2.3,0.955114,"{'22aa20b0': 0.0206926, '22ab20a0': 0.1642669,...",1.0,2Π,35877.8
9,3,Doublet,2,2.2,-661.003948,-661.025732,10,0.0,0.0,1.420276,-660.775209,1.420276,2.2,0.955114,"{'22aab020': 0.0206926, '22aba020': 0.1642669,...",1.0,2Π,35877.8
1,1,Doublet,1,2.1,-660.987386,-661.019217,10,0.0,0.0,-1.960257,-660.748771,1.960257,2.1,0.941384,"{'22a02020': 0.0645692, '220a2020': 0.8081035,...",0.0,2Σ+,37307.5
2,1,Doublet,1,3.1,-660.968666,-660.993563,10,0.0,0.0,0.760857,-660.730059,0.760857,3.1,0.950590,"{'22a02020': -0.090828, '220a2020': -0.3384821...",0.0,2Σ+,42938.0
6,2,Doublet,1,4.1,-660.934526,-660.952830,10,0.0,0.0,1.382673,-660.718892,1.382673,4.1,0.960083,"{'22a020ba': 0.585081, '22a0ba20': -0.5850799,...",2.0,2Δ,51877.9
18,5,Doublet,4,1.4,-660.934526,-660.952829,10,0.0,0.0,1.382691,-660.718892,1.382691,1.4,0.960084,"{'22a02ab0': 0.6757897, '22a02ba0': -0.0003395...",2.0,2Δ,51878.0
20,6,Doublet,4,2.4,-660.932190,-660.950473,10,0.0,0.0,1.394746,-660.716449,1.394746,2.4,0.960145,"{'22a02ab0': 0.67567, '22a02ba0': -0.0144656, ...",0.0,2Σ-,52395.2


In [392]:
# check for erroneously repeated CI roots
tol = 1.e-6
for irrep, grp in dfci.sort_values('Energy').groupby(['Spin', 'Irrep']):
    e = grp.Energy.values
    de = e[1:] - e[:-1]
    smal = np.abs(de) < tol
    for i, s in enumerate(smal):
        if s:
            print('Warning: closely repeated root in MRCI')
            display(grp.iloc[[i,i+1]])
print('Checking for discrepancies between reference energy and CASSCF energy')
dfcheck = dfci[['Spin', 'Irrep', 'Label', 'Edav', 'Eref']].copy()
ecas = []
for i, row in dfcheck.iterrows():
    ecas.append(CAS.results[(CAS.results.Label == row.Label) & (CAS.results.Spin == row.Spin)].Energy.values[0])
dfcheck['CAS'] = ecas
dfcheck['diff'] = np.round(dfcheck.CAS - dfcheck.Eref, 6)
dfbad = dfcheck[np.abs(dfcheck['diff']) > 0.1].sort_values(['Spin', 'Irrep', 'Label'])
if len(dfbad):
    display(dfbad)
else:
    print('\t--looks good')

Checking for discrepancies between reference energy and CASSCF energy
	--looks good


In [393]:
# read SO-CI
SOCI = mpr.fullmatSOCI(fsoci, hybrid=True)
dfterms = SOCI.average_terms()

Computational group = C2v
CASSCF states:
    22 Doublet
Replacing MRCI+Q energies by HLSDIAG values
*** Warning: "44 SO states but only 36 unique basis states" in parse_SOmatrix()


In [397]:
# Build handling for pairs of basis states
sibbs = []  # sibling basis state that differs only in sign of Sz
for bas in SOCI.basis:
    for jbas in range(SOCI.dimen):
        obas = SOCI.basis[jbas]
        if (bas[0] == obas[0]) and (bas[1] == obas[1]) and (bas[2] == -obas[2]):
            sibbs.append(jbas)
            break
def wt_avail(ibas, omega):
    # Return the weight available from BS for specified omega
    #   sum of used weights for BS, across omegas, cannot exceed 1.0
    #   sum of used weights for omega, across sibling pairs, cannot exceed 1.0
    totused = sum([used for om, used in bas_omega[ibas].items()])
    try:
        wtleft = 1 - bas_omega[ibas][omega]
    except KeyError:
        # this omega not available for this basis state
        return 0
    # deduct any used by sibling
    jbas = sibbs[ibas]
    wtleft -= bas_omega[jbas][omega]
    # don't exceed totused
    wtleft = min(wtleft, 1-totused)

In [398]:
sibbs

[6,
 7,
 8,
 9,
 10,
 11,
 0,
 1,
 2,
 3,
 4,
 5,
 14,
 15,
 12,
 13,
 21,
 22,
 23,
 24,
 25,
 16,
 17,
 18,
 19,
 20,
 31,
 32,
 33,
 34,
 35,
 26,
 27,
 28,
 29,
 30,
 38,
 39,
 36,
 37,
 42,
 43,
 40,
 41]

In [394]:
SOCI.basis

[('1.1', 0.5, 0.5),
 ('2.1', 0.5, 0.5),
 ('3.1', 0.5, 0.5),
 ('5.1', 0.5, 0.5),
 ('7.1', 0.5, 0.5),
 ('8.1', 0.5, 0.5),
 ('1.1', 0.5, -0.5),
 ('2.1', 0.5, -0.5),
 ('3.1', 0.5, -0.5),
 ('5.1', 0.5, -0.5),
 ('7.1', 0.5, -0.5),
 ('8.1', 0.5, -0.5),
 ('4.1', 0.5, 0.5),
 ('6.1', 0.5, 0.5),
 ('4.1', 0.5, -0.5),
 ('6.1', 0.5, -0.5),
 ('1.2', 0.5, 0.5),
 ('2.2', 0.5, 0.5),
 ('3.2', 0.5, 0.5),
 ('4.2', 0.5, 0.5),
 ('5.2', 0.5, 0.5),
 ('1.2', 0.5, -0.5),
 ('2.2', 0.5, -0.5),
 ('3.2', 0.5, -0.5),
 ('4.2', 0.5, -0.5),
 ('5.2', 0.5, -0.5),
 ('1.3', 0.5, 0.5),
 ('2.3', 0.5, 0.5),
 ('3.3', 0.5, 0.5),
 ('4.3', 0.5, 0.5),
 ('5.3', 0.5, 0.5),
 ('1.3', 0.5, -0.5),
 ('2.3', 0.5, -0.5),
 ('3.3', 0.5, -0.5),
 ('4.3', 0.5, -0.5),
 ('5.3', 0.5, -0.5),
 ('1.4', 0.5, 0.5),
 ('3.4', 0.5, 0.5),
 ('1.4', 0.5, -0.5),
 ('3.4', 0.5, -0.5),
 ('2.4', 0.5, 0.5),
 ('4.4', 0.5, 0.5),
 ('2.4', 0.5, -0.5),
 ('4.4', 0.5, -0.5)]

In [417]:
def SO_assign_omega2(SOCI, csq_thresh=0.001, silent=False,
                    ordering='up', failure='crash', debug=False):
    '''
    New attempt (May 2023) to assign Omega to SO-CI states
        Based upon averaged terms instead of MRCI states
    Assign 0+/0- simply based upon irrep (if C2v)
    'SOCI' is a mpr.fullmatSOCI() object
    'ordering' determines which order Omega values are considered
        ('up', 'down', 'rare', 'common')
    'failure' == 'OK' will continue despite assignment failure
    Return a DataFrame
    '''
    sob_ici, ci_sob = mpr.link_MRCI_SObasis(SOCI.mrci, SOCI.basis)
    vals = SOCI.SOe.results.Eshift.values  # cm-1 energies relative to the reference
    Nrs = SOCI.SOe.results.Nr.values
    E = SOCI.SOe.results.E.values  # state energies in hartree
    nirrep = len(set(SOCI.SOe.results.Irrep))
    dimen = len(SOCI.basis)
    SO_omega = [None] * dimen   # assigned values of Omega
    parenterm = [None] * dimen  # term label for dominant parent
    termlabel = []              # term label including omega
    leadwt = [0] * dimen        # weight of leading term
    # 
    # get list of required omega values
    omegavals = mpr.omega_counts(mrci, silent=True)
    if not silent:
        print('Target omega counts:', omegavals)
    #
    # determine the possible omega values for each SO basis state
    bas_omega = []  # weights already used for possible omega values, for each basis state
    omall = set()   # possible omegas across whole SO basis 
    for ibas, bas in enumerate(SOCI.basis):
        Sz = bas[2]
        Lz = mrci[sob_ici[ibas]].Lz
        lo = round(abs(Sz-Lz), 1)
        hi = round(abs(Sz+Lz), 1)
        omset = set((lo, hi))
        omall = omall.union(omset)
        # before assignments, used weights are all zero
        bas_omega.append({o: 0. for o in omset})
        jci = sob_ici[ibas]
    #
    # Keep track of omega values assigned for each averaged term
    #    and for each basis state   
    # Setup accounting within terms 
    dfterm = SOCI.dfterm.copy()
    omavail = [{om: 0. for om in omall} for i in range(SOCI.nterm)]
    dfterm['om_avail'] = omavail
    term_idx = [-1] * dimen  # index/row of average term corresponding to SO basis state
    for ibas in range(dimen):
        jterm = SOCI.sob_iterm[ibas]
        term = dfterm.at[jterm, 'Term']
        omd = dfterm.loc[dfterm.Term == term, 'om_avail'].values[0]
        for om in bas_omega[ibas]:
            omd[om] += 1
        iterm = dfterm[dfterm.Term == term].index.values[0]
        term_idx[ibas] = iterm
    # Correct double-counting for spatially degenerate terms
    for i, row in dfterm.iterrows():
        n = len(row.idx)
        if n > 1:
            omd = row.om_avail
            for om in omd.keys():
                omd[om] /= n
    # check for inconsistent omega counting (terms vs. basis states)
    twtsum = 0
    wtsum = 0
    for om_avail in dfterm.om_avail:
        for om, n in om_avail.items():
            twtsum += n
    for om_avail in bas_omega:
        for om, n in om_avail.items():
            wtsum += n
    if debug:
        print(f'Initial weight sum for terms = {twtsum:.3f}')
        print(f'Initial weight sum for basis states = {wtsum:.3f}')
    if twtsum != wtsum:
        # something is wrong; show term components
        for iterm, row in dfterm.iterrows():
            tsum = sum(row.om_avail.values())
            bsum = 0
            for ibas in SOCI.term_iso[iterm]:
                bsum += sum(bas_omega[ibas].values())
            if tsum != bsum:
                print(f'iterm = {iterm} has total {tsum}', chem.round_dict(row.om_avail))
                for ibas in SOCI.term_iso[iterm]:
                    sum1 = sum(bas_omega[ibas].values())
                    print(f'iterm = {iterm}, ibas = {ibas} has total {sum1}:', chem.round_dict(bas_omega[ibas]))
    sibbs = []  # sibling basis state that differs only in sign of Sz
    for bas in SOCI.basis:
        for jbas in range(dimen):
            obas = SOCI.basis[jbas]
            if (bas[0] == obas[0]) and (bas[1] == obas[1]) and (bas[2] == -obas[2]):
                sibbs.append(jbas)
                break

    # look at decreasingly smaller contributions
    omposs = [omall.copy() for i in range(dimen)]  # possible omega values for each SO state (list of sets)
    ncontrib = 0   # depth of contributions to consider
    changed = True
    above_thresh = True
    nleft = dimen  # number of unassigned SO states
    omegavalslist = list(omegavals.keys())
    if ordering == 'up':
        omegavalslist = sorted(omegavalslist)
    elif ordering == 'down':
        omegavalslist = sorted(omegavalslist, reverse=True)
    elif ordering == 'rare':
        omegavalslist = sorted(omegavalslist, key=lambda x: omegavals[x])
    elif ordering == 'common':
        omegavalslist = sorted(omegavalslist, key=lambda x: omegavals[x], reverse=True)
    else:
        chem.print_err('', f'Unknown ordering "{ordering}"')
        
    def assign_one(istate, omega):
        # update vars to assign omega value to SO state
        nonlocal changed
        SO_omega[istate] = omega
        omegavals[omega] -= 1
        # deduct from term's om_avail[omega]
        for iterm, row in dfterm.iterrows():
            twt = SOCI.termwt[iterm, istate] # weight of term in this state
            if debug and (twt > csq_thresh):
                print(f'\ttwt = {twt:.3f} for iterm = {iterm}')
            omd = row.om_avail
            omd[omega] -= twt
            # check for deficits
            if debug and (omd[omega] < -csq_thresh):
                print(f'\tnegative omd[{omega}] {omd[omega]:.5f} for term {iterm}')
        # add to basis state's bas_omega[omega] (showing usage, not remaining)
        for ibas in range(dimen):
            wt = SOCI.vecsq[ibas, istate]
            if debug and (wt > csq_thresh):
                print(f'\twt = {wt:.3f} for basis state #{ibas}')
            omd = bas_omega[ibas]
            try:
                omd[omega] += wt
                if debug and (omd[omega] > 1 + csq_thresh):
                    print(f'\texcessive omd[{omega}] {omd[omega]:.5f} for basis state {ibas}')
            except KeyError:
                # invalid omega for this basis state
                pass
        changed = True
        return
    def wt_avail(ibas, omega):
        # Return the weight available from BS for specified omega
        #   sum of used weights for BS, across omegas, cannot exceed 1.0
        #   sum of used weights for omega, across sibling pairs, cannot exceed 1.0
        try:
            wtleft = 1 - bas_omega[ibas][omega]
        except KeyError:
            # this omega not available for this basis state
            return 0
        # deduct any used by sibling
        jbas = sibbs[ibas]
        wtleft -= bas_omega[jbas][omega]
        # don't exceed totused
        totused = sum([used for om, used in bas_omega[ibas].items()])
        wtleft = min(wtleft, 1-totused)
        return wtleft
    def thresh_filter(ibas, thresh):
        # Return a dict of {omega: remaining_weight} restricted to 
        #   remaining_weight >= thresh
        d = {}
        for om in bas_omega[ibas].keys():
            wa = wt_avail(ibas, om)
            if wa > thresh:
                d[om] = wa
        return d
    def thresh_tfilter(iterm, thresh):
        # Return a dict of {omega: remaining_weight} restricted to 
        #   remaining_weight >= thresh
        d = {k: v for k, v in dfterm.at[iterm, 'om_avail'].items() if v >= thresh}
        return d
    
    while (changed or above_thresh) and nleft:
        changed = False
        above_thresh = False
        
        #### Are there omega values for which number possible = number needed?
        for om in omegavalslist:
            need = omegavals[om]
            iposs = []  # list of states that could assign Omega = 'om'
            for iso, oms in enumerate(omposs):
                if (om in oms) and (SO_omega[iso] is None):
                    iposs.append(iso)
            if debug:
                s = 'for Ω = {:.1f}, need {:d} states, {:d} possibilities'
                print(s.format(om, need, len(iposs)))
            if (need > 0) and (len(iposs) == need):
                # assign these
                for iso in iposs:
                    assign_one(iso, om)
                need = 0
                if debug:
                    print('\t--assigned_A')
            if need == 0:
                # don't assign any more states to this Omega
                for oms in omposs:
                    oms.discard(om)
                if debug and len(iposs):
                    print(f'\teliminating omega = {om} as a possibility')
                    
        #### Consider the next smaller contribution to each SO state
        for iso in range(dimen):
            # loop over SO states, not basis functions
            if SO_omega[iso] is not None:
                # already assigned
                continue
            if debug:
                print(f'Weight[{ncontrib}] in state {iso}')
            wt = SOCI.vecsq[:, iso]
            twt = SOCI.termwt[:, iso]
            # sort the index   
            idx = np.argsort(-wt)  # decreasing order
            ibas = idx[ncontrib]
            iterm = SOCI.sob_iterm[ibas] # parent term of this basis state
            if ncontrib == 0:
                # First pass--get term symbol from term with largest total weight
                #   it might not be the same as the basis state with the largest weight?
                itmax = np.argmax(twt)
                parenterm[iso] = dfterm.at[itmax, 'Term']
                leadwt[iso] = twt[itmax]
                if debug:
                    print('\tleading term is {:s} with iterm = {:d}'.format(parenterm[iso], itmax))
                    print(f'\tleading basis state is #{ibas} with usage', chem.round_dict(bas_omega[ibas]),
                         f'and sib {sibbs[ibas]}')
                    if ibas not in SOCI.term_iso[itmax]:
                        print('\t\t---the leading basis state is not part of the leading term---')
            if wt[ibas] < csq_thresh:
                # done with significant weights for this state
                continue
            else:
                above_thresh = True
            print('>>>before ', bas_omega[ibas])
            bas_rem = thresh_filter(ibas, wt[ibas])  # dict of omegas and remaining weights that are large enough
            print('>>>after ', bas_rem)
            u = omposs[iso].intersection(bas_rem.keys())
            if debug:
                print('\tweight {:.4f} for BS#{} in term {} with remaining possibilities {}'.format(wt[ibas], 
                                            ibas, dfterm.at[iterm, 'Term'], chem.round_dict(bas_rem)))
            if len(u) == 0:
                # all possibilities eliminated! dismiss the last component as noise
                if debug:
                    print('All possibilities eliminated! Dismiss this weight as noise')
                continue
            omposs[iso] = u.copy()
            if debug:
                print('\tpossible:', omposs[iso])
            if len(u) == 1:
                # only one omega remains; assign it
                om = u.pop()
                assign_one(iso, om)
                if debug:
                    print('\t--assigned_B')
        ncontrib += 1
        nleft = len([x for x in SO_omega if x is None])
    #
    # check for states missing assignments
    for i, om in enumerate(SO_omega):
        if om is None:
            # this is bad
            print('missing Ω for SO state {:d} with E = {:.1f} cm-1 and parent {:s} '.format(i, vals[i], parenterm[i]))
            print('\tpossibilities: ', omposs[i])
            # install the ambiguity in the list
            SO_omega[i] = repr(omposs[i])
    # check for excess and lacking omega counts
    ok = True
    for om, count in omegavals.items():
        if count < 0:
            s = f'Overcounting for Ω = {om}'
            chem.print_err('', s, halt=False)
            ok = False
            if failure != 'OK':
                raise ValueError(s)  # increasing the threshold may help
        if count > 0:
            chem.print_err('', f'Missing {count} states for Ω = {om}', halt=False)
            ok = False
    # Find largest and smallest term residua
    residmax = [-1, '?', -1, -np.inf]  # [iterm, term label, omega, residuum]
    residmin = [-1, '?', -1,  np.inf]
    residsum = {o: 0 for o in omall}   # sums across all terms
    for iterm, row in dfterm.iterrows():
        for om, res in row.om_avail.items():
            if res > residmax[3]:
                residmax = [iterm, row.Term, om, res]
            if res < residmin[3]:
                residmin = [iterm, row.Term, om, res]
            residsum[om] += res
    # Likewise for basis states
    bmax = [-1, -1, -np.inf]  # [ibas, omega, residuum]
    bmin = [-1, -1, np.inf]
    bsum = {o: 0 for o in omall}  # sums across all basis states
    for ibas in range(dimen):
        for om, res in bas_omega[ibas].items():
            if res > bmax[2]:
                bmax = [ibas, om, res]
            if res < bmin[2]:
                bmin = [ibas, om, res]
            bsum[om] += res
    # Round the totals for printing
    for om in omall:
        residsum[om] = np.round(residsum[om], 3)
        bsum[om] = np.round(bsum[om], 3)
    if debug:
        print('Extremes of term residua:')
        print('\tmax for iterm = {:d}, {:s}, omega = {:.1f} residuum = {:.4f}'.format(*residmax))
        print('\tmin for iterm = {:d}, {:s}, omega = {:.1f} residuum = {:.4f}'.format(*residmin))
        print('\tSums of residua across terms:', residsum)
        print('Extremes of basis-state residua')
        print('\tmax for ibas = {:d}, omega = {:.1f} resid = {:.4f}'.format(*bmax))
        print('\tmin for ibas = {:d}, omega = {:.1f} resid = {:.4f}'.format(*bmin))
        print('\tSums of residua across basis states:', bsum)
    if not ok:
        print('*******************************')
        print('*** OMEGA ASSIGNMENTS FAILED***')
        print('*******************************')
        # set all Omega=0 to prevent pairing
        SO_omega = [0] * len(SO_omega)
    #
    # strip any unnecessary '(1)' specifiers from MRCI term labels
    just_one = set()
    rx = re.compile('\((\d+)\)(\S+)')
    for t in parenterm:
        m = rx.match(t)
        if m:
            if int(m.group(1)) == 1:
                just_one = just_one.union({m.group(2)})
            else:
                # prefix is higher than (1); keep prefixes for this term symbol
                just_one.discard(m.group(2))
    for symb in just_one:
        # remove leading (1)
        for i, t in enumerate(parenterm):
            if t == '(1)' + symb:
                parenterm[i] = symb
    # create state labels that include omega as a "subscript"
    for i, pt in enumerate(parenterm):
        try:
            olbl = mpr.halves(SO_omega[i])
        except TypeError:
            # omega assignment failed
            olbl = SO_omega[i]
        if (SO_omega[i] == 0) and (nirrep == 4):
            # assign parity based upon irrep (assuming C2v!)
            irr0 = SOCI.SOe.results.at[i, 'Irrep']
            if irr0 == 1:
                olbl = olbl + '+'
            elif irr0 == 4:
                olbl = olbl + '-'
            elif not silent:
                print('Omega = 0 but irrep = {:d}'.format(irr0))
        termlabel.append('_'.join([pt, olbl]))
    # return a DataFrame
    df = pd.DataFrame({'E': E, 'cm-1': vals, mpr.OMEGA: SO_omega, 'term': parenterm, 
                       'wt': leadwt, 'label': termlabel, 'Nr': Nrs})
    df['exc'] = df['cm-1'] - df['cm-1'].min()
    return df, ok
##


In [418]:
    self = SOCI  
    csq_thresh = 0.001
    silent = False
    ordering = 'up'
    failure = 'crash'
    debug = True
    dfstates, ok = SO_assign_omega2(self, csq_thresh=csq_thresh,
                                    silent=silent, ordering=ordering, failure=failure, debug=debug)

Target omega counts: {0.5: 26, 1.5: 14, 2.5: 4}
Initial weight sum for terms = 44.000
Initial weight sum for basis states = 0.000
iterm = 0 has total 2.0 {0.5: 2.0, 1.5: 0.0, 2.5: 0.0}
iterm = 0, ibas = 0 has total 0.0: {0.5: 0.0}
iterm = 0, ibas = 6 has total 0.0: {0.5: 0.0}
iterm = 1 has total 4.0 {0.5: 2.0, 1.5: 2.0, 2.5: 0.0}
iterm = 1, ibas = 16 has total 0.0: {0.5: 0.0, 1.5: 0.0}
iterm = 1, ibas = 21 has total 0.0: {0.5: 0.0, 1.5: 0.0}
iterm = 1, ibas = 26 has total 0.0: {0.5: 0.0, 1.5: 0.0}
iterm = 1, ibas = 31 has total 0.0: {0.5: 0.0, 1.5: 0.0}
iterm = 2 has total 4.0 {0.5: 2.0, 1.5: 2.0, 2.5: 0.0}
iterm = 2, ibas = 17 has total 0.0: {0.5: 0.0, 1.5: 0.0}
iterm = 2, ibas = 22 has total 0.0: {0.5: 0.0, 1.5: 0.0}
iterm = 2, ibas = 27 has total 0.0: {0.5: 0.0, 1.5: 0.0}
iterm = 2, ibas = 32 has total 0.0: {0.5: 0.0, 1.5: 0.0}
iterm = 3 has total 2.0 {0.5: 2.0, 1.5: 0.0, 2.5: 0.0}
iterm = 3, ibas = 1 has total 0.0: {0.5: 0.0}
iterm = 3, ibas = 7 has total 0.0: {0.5: 0.0}
iterm = 4 

In [331]:
Compos = mpr.readSOcompos(fsoci)[0][-1]
Compos.basis

,CI lbl,Spin,Sz
Nr,,,
1,1.1,Doublet,0.5
2,1.1,Doublet,-0.5
3,1.2,Doublet,0.5
4,2.2,Doublet,0.5
5,1.2,Doublet,-0.5
6,2.2,Doublet,-0.5


In [332]:
len(Compos.basis)

6

In [333]:
len(set(Compos.basis['CI lbl']))

3

In [416]:
np.set_printoptions(threshold=sys.maxsize)
np.round(SOCI.vecsq,3)

array([[1.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.006,
        0.   , 0.994, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.002,
        0.   , 0.   , 0.   , 0.   , 0.998, 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0. 

In [335]:
SOCI.basis

[('1.1', 0.5, 0.5),
 ('1.1', 0.5, -0.5),
 ('1.2', 0.5, 0.5),
 ('2.2', 0.5, 0.5),
 ('1.2', 0.5, -0.5),
 ('2.2', 0.5, -0.5)]

In [51]:
thr = 0.001
for iso in [2, 3]:
    print(f'iso = {iso}')
    vec = SOCI.vec[:, iso] 
    wt = SOCI.vecsq[:, iso]
    twt = SOCI.termwt[:, iso]  # averaged terms
    for i, bas in enumerate(SOCI.basis):
        if wt[i] < thr:
            continue
        jterm = SOCI.sob_iterm[i]
        #print('\t', i, bas, '\t', jterm, SOCI.termlabel_index(jterm)[0], f'vec = {np.round(vec[i], 3)}')
        print('\t', i, bas, '\t', jterm, SOCI.termlabel_index(jterm)[0], f'wt = {np.round(wt[i], 3)}')

iso = 2
	 5 ('2.1', 0.5, -0.5) 	 1 (1)2Σ+ wt = 0.687
	 22 ('1.2', 0.5, 0.5) 	 2 (1)2Π wt = 0.148
	 38 ('1.3', 0.5, 0.5) 	 2 (1)2Π wt = 0.149
	 85 ('1.2', 1.5, 0.5) 	 5 (1)4Π wt = 0.001
	 91 ('1.2', 1.5, -1.5) 	 5 (1)4Π wt = 0.003
	 101 ('1.3', 1.5, 0.5) 	 5 (1)4Π wt = 0.001
	 107 ('1.3', 1.5, -1.5) 	 5 (1)4Π wt = 0.003
	 118 ('2.4', 1.5, -0.5) 	 8 (1)4Σ- wt = 0.001
iso = 3
	 0 ('2.1', 0.5, 0.5) 	 1 (1)2Σ+ wt = 0.687
	 28 ('1.2', 0.5, -0.5) 	 2 (1)2Π wt = 0.148
	 44 ('1.3', 0.5, -0.5) 	 2 (1)2Π wt = 0.149
	 82 ('1.2', 1.5, 1.5) 	 5 (1)4Π wt = 0.003
	 88 ('1.2', 1.5, -0.5) 	 5 (1)4Π wt = 0.001
	 98 ('1.3', 1.5, 1.5) 	 5 (1)4Π wt = 0.003
	 104 ('1.3', 1.5, -0.5) 	 5 (1)4Π wt = 0.001
	 116 ('2.4', 1.5, 0.5) 	 8 (1)4Σ- wt = 0.001


In [52]:
for iso in [4, 5]:
    print(f'iso = {iso}')
    vec = SOCI.vec[:, iso] 
    wt = SOCI.vecsq[:, iso]
    twt = SOCI.termwt[:, iso]  # averaged terms
    for i, bas in enumerate(SOCI.basis):
        if wt[i] < thr:
            continue
        jterm = SOCI.sob_iterm[i]
        #print('\t', i, bas, '\t', jterm, SOCI.termlabel_index(jterm)[0], f'vec = {np.round(vec[i], 3)}')
        print('\t', i, bas, '\t', jterm, SOCI.termlabel_index(jterm)[0], f'wt = {np.round(wt[i], 3)}')

iso = 4
	 15 ('1.1', 0.5, -0.5) 	 0 (1)2Δ wt = 0.138
	 22 ('1.2', 0.5, 0.5) 	 2 (1)2Π wt = 0.353
	 24 ('4.2', 0.5, 0.5) 	 12 (3)2Π wt = 0.001
	 38 ('1.3', 0.5, 0.5) 	 2 (1)2Π wt = 0.353
	 40 ('4.3', 0.5, 0.5) 	 12 (3)2Π wt = 0.001
	 63 ('1.4', 0.5, -0.5) 	 0 (1)2Δ wt = 0.138
	 97 ('2.2', 1.5, -1.5) 	 9 4Φ wt = 0.002
	 113 ('2.3', 1.5, -1.5) 	 9 4Φ wt = 0.002
	 114 ('2.4', 1.5, 1.5) 	 8 (1)4Σ- wt = 0.005
iso = 5
	 10 ('1.1', 0.5, 0.5) 	 0 (1)2Δ wt = 0.138
	 28 ('1.2', 0.5, -0.5) 	 2 (1)2Π wt = 0.353
	 30 ('4.2', 0.5, -0.5) 	 12 (3)2Π wt = 0.001
	 44 ('1.3', 0.5, -0.5) 	 2 (1)2Π wt = 0.353
	 46 ('4.3', 0.5, -0.5) 	 12 (3)2Π wt = 0.001
	 58 ('1.4', 0.5, 0.5) 	 0 (1)2Δ wt = 0.138
	 94 ('2.2', 1.5, 1.5) 	 9 4Φ wt = 0.002
	 110 ('2.3', 1.5, 1.5) 	 9 4Φ wt = 0.002
	 120 ('2.4', 1.5, -1.5) 	 8 (1)4Σ- wt = 0.005


In [53]:
if not ok:
    thresh = 2.e-5 # for energies
    wthr = 1.e-3   # for leading-term weights
    pd.set_option('display.max_rows', 500)
    print('*** Attempt to assign Omegas by interpolation!  ***')
    print('--- Using energy only ---')
    datfile = r'C:\Users\irikura\OneDrive - NIST\Karl\PtH_anion\guiding_scan\guiding_pots.tsv'
    print(f'Using PEC data file {datfile}')
    dfPEC = pd.read_csv(datfile, sep='\t')
    # expect a column 'R' followed by columns with state labels
    # generate interpolations
    from collections import Counter
    Einterp = []
    Ominterp = []
    re_om = re.compile('(\d)[-+]?$')
    x = dfPEC.R.values
    states = []
    for st in dfPEC.columns[1:]:
        y = dfPEC[st].values
        fPEC = chem.fit_diatomic_potential(x, y)
        E = float(fPEC(R))
        m = re_om.search(st)
        Om = int(m.group(1))
        states.append(st)
        Einterp.append(E)
        Ominterp.append(Om)
    dfinterp = pd.DataFrame({'Label': states, 'Om': Ominterp, 'E': Einterp}).sort_values('E').reset_index(drop=True)
    # add column for energy increments
    evals = dfinterp.E.values
    incr = [np.nan] + list(evals[1:] - evals[:-1])
    dfinterp['incrE'] = np.round(incr, 6)
    dfinterp['used'] = False
    print('\nInterpolated states (expectation):')
    display(dfinterp)
    icount = {}
    for om, v in Counter(dfinterp.Om).items():
        if om == 0:
            icount[om] = v
        else:
            icount[om] = v * 2
    icount = Counter(icount)
    print('Target (correct) counts:', icount)
    
    # Do not re-sort dfso by energy because ordering of degen levels can get scrambled
    dfso = SOCI.dfso.sort_values('E').reset_index(drop=True)  # to be assigned
    dfso['Ω'] = None
    dfso['label'] = None
    dfso['Olbl'] = None
    evals = dfso.E.values
    incr = [np.nan] + list(evals[1:] - evals[:-1])
    dfso['incrE'] = np.round(incr, 6)
    wts = dfso.wt.values
    dwt = [np.nan] + list(wts[1:] - wts[:-1])
    dfso['wtdiff'] = np.round(np.abs(dwt), 5)
    nstates = len(dfso)
    print('\nActual states to be assigned:')
    display(dfso)
   
    # Since usual Omega assignment failed, do not assume that the term symbols are reliable
    def assignstate():
        # use globals
        dfso.loc[i, 'Olbl'] = jow.Label
        dfso.loc[i, 'Ω'] = jow.Om
        dfinterp.loc[j, 'used'] = True
        print(f'assign state {i} with {jow.Label}')
        if jow.Om > 0:
            # also assign its twin (should be the next state by energy)
            dfso.loc[i+1, 'Olbl'] = jow.Label
            dfso.loc[i+1, 'Ω'] = jow.Om                    
            print(f'\tpair {i+1} with {jow.Label}')
        return

    print('Start with biggest gaps:')
    # look for big gaps, as more reliable than small gaps
    dfinterp = dfinterp.sort_values('incrE', ascending=False)
    for i, row in dfso.sort_values('incrE', ascending=False).iterrows():
        if row.incrE < 2 * thresh:
            # too small to trust
            continue
        #display(row.to_frame().T)
        for j, jow in dfinterp.iterrows():
            if jow.incrE < 2 * thresh:
                # gap too small to trust
                continue
            if jow.used:
                # already matched to an actual state
                continue
            if abs(row.incrE - jow.incrE) < thresh:
                de = abs(row.E - jow.E)
                if de < thresh:
                    # this looks like a match; check for accidental degeneracy
                    #if (i+1 < nstates) and (dfso.loc[i+1, 'incrE'] < thresh):
                    #    # next level is close; it might be the twin
                    #    if (i+2 < nstates) and (dfso.loc[i+2, 'incrE'] > thresh):
                    #        # looks like an accidental degeneracy (3 levels)
                    #        break
                    # this is a match; assign it
                    assignstate()
    dfinterp = dfinterp.sort_values('E')
    display(dfso[dfso.Ω.isnull()])
    counts = Counter(dfso.Ω)
    print('current counts:', counts)
    print('missing counts:', icount - counts)
    
    # drop used levels from dfinterp
    dfinterp = dfinterp[dfinterp.used == False]

    print('\nLook for close energy matches:')
    for i, row in dfso.iterrows():
        if row.Ω is not None:
            # already assigned
            continue
        for j, jow in dfinterp.iterrows():
            if jow.used:
                # row already matched
                continue
            dE = abs(row.E - jow.E)
            if dE < thresh:
                # a match
                assignstate()
    display(dfso[dfso.Ω.isnull()])
    counts = Counter(dfso.Ω)
    print('current counts:', counts)
    print('missing counts:', icount - counts)
    dfinterp = dfinterp[dfinterp.used == False]

    if (len(dfinterp) == 0) and (sum((icount - counts).values()) == 0):
        print('\nAll states assigned!')
        # drop unneeded columns
        dfdups = dfso[['E', 'cm-1', 'Ω', 'exc', 'Olbl']]
        df = mpr.average_SO_levels(dfdups, be_same=['Ω'])
        #df['Olbl'] = chem.enumerative_prefix([s.split('_')[1] for s in df.label])
        display(df)
        # copy into the SOCI object
        SOCI.dfso = df

In [54]:
dfterms['Ecm'] = np.round((dfterms.Edav - dfterms.Edav.min()) * chem.AU2CM, 1)
dfterms

,Term,dipZ,Edav,idx,ecm,Ecm
0,(1)2Δ,-0.588172,-119.784064,"[29, 5]",0.0,0.0
1,(1)2Σ+,-0.686412,-119.783698,[0],80.5,80.5
2,(1)2Π,-0.652485,-119.777318,"[11, 19]",1480.7,1480.7
3,(1)4Δ,-0.161305,-119.730043,"[48, 36]",11856.4,11856.4
4,(2)2Σ+,-0.014132,-119.727746,[1],12360.4,12360.4
5,(1)4Π,-0.096289,-119.727039,"[42, 38]",12515.7,12515.7
6,(2)2Δ,0.101961,-119.718361,"[30, 6]",14420.2,14420.2
7,(2)2Π,0.095556,-119.716350,"[12, 20]",14861.5,14861.5
8,(1)4Σ-,-0.142579,-119.715854,[46],14970.4,14970.4
9,4Φ,-0.110737,-119.714348,"[41, 45]",15301.1,15301.1


In [55]:
# Is this a hybrid calculation (prepared by build_hybrid_soci_input.ipynb)?
ccterms = []  # list of input CCSD(T) terms
rx_hyb = re.compile('HLSDIAG\(.+\s+!\s*.*(ccsd|anchored|shifted|input)')
is_hybrid = False
with open(fsoci, 'r', encoding='utf8') as F:
    for line in F:
        if rx_hyb.search(line):
            is_hybrid = True
            if 'input' in line:
                # an anchor term; extract its label
                words = line.split()
                if words[2] not in ccterms:
                    ccterms.append(words[2])

In [56]:
# transpose absolute energies and copy to clipboard
dfcp = SOCI.dfso[['Olbl', 'E', '<i|z|i>']].copy().sort_values('Olbl').set_index('Olbl')
dfcp.rename(columns={'E': f'{R}'}, inplace=True)   # put bond length in that position for pasting to Excel
dfcp.T.to_clipboard()
if is_hybrid:
    print(f'Hybrid SO-CI energies for R={R} copied to clipboard, for pasting into Excel')
else:
    print(f'Standard SO-CI energies for R={R} copied to clipboard, for pasting into Excel')

AttributeError: 'fullmatSOCI' object has no attribute 'dfso'

In [ ]:
dfcp

### Term composition of some levels

In [ ]:
olabels = ['(1)1/2', '(1)3/2', '(1)5/2']
for olabel in olabels:
    ilev = SOCI.dfso[SOCI.dfso.Olbl == olabel].index[0]
    #print('ilvel =', ilev)
    print('Composition of level #{:d}:  "{:s}" or "{:s}"'.format(ilev, SOCI.dfso.loc[ilev].Olbl,
                                                             SOCI.dfso.loc[ilev].label))
    dfci, dfterm = SOCI.composition_of_level(ilev, thr=1.e-6, normalize=True)
    wtcol = dfterm.columns[-1]
    display(dfterm.sort_values(wtcol, ascending=False))
    print('Sum of weights = {:.3f}'.format(dfterm[wtcol].sum()))
    ccsum = dfterm[dfterm.Term.isin(ccterms)][wtcol].sum()
    print('Sum of weights from CC terms = {:.3f}\n'.format(ccsum))

### Distribution of a term among levels

In [ ]:
#term = '(1)1Σ+'
term = '(1)2Δ'
if is_hybrid:
    typ = 'hybrid'
else:
    typ = 'standard'
print('Distribution of term "{:s}" among {:s} levels'.format(term, typ))
print(f'R = {R}')
df = SOCI.level_contributions_from_term(term, thr=1.e-6, normalize=True)
df['prod'] = np.round(df.exc * df[term], 1)
ebar = df['prod'].sum() / df[term].sum()
print('Weighted mean energy of {:s} = {:.1f} cm-1'.format(term, ebar))
#display(df.sort_values(term, ascending=False))
display(df[df[term] > 0.0005].sort_values('E').style.format({term: '{:.4f}'}))
print('Total weight = {:.4f}'.format(df[term].sum()))

### Contribution of CCSD(T) terms to all levels

In [ ]:
# Get contributions of CC terms to all levels
# Loop over levels to get the normalization right
dfcc = SOCI.dfso.copy()
ccsums = []
for ilev in dfcc.index:
    dfci, dfterm = SOCI.composition_of_level(ilev, thr=1.e-6,
                                normalize=True, silent=True)
    wtcol = dfterm.columns[-1]
    ccsum = dfterm[dfterm.Term.isin(ccterms)][wtcol].sum()
    ccsums.append(ccsum)
dfcc['CCwt'] = ccsums
fmt = {chem.OMEGA: '{:.1f}', 'exc': '{:.2f}', 'CCwt': '{:.3f}'}
display(dfcc[dfcc.columns[2:]].style.format(fmt))
print('Total of CCwt column = {:.3f}'.format(dfcc.CCwt.sum()))
print('The CC terms are', ccterms)

In [ ]:
df = SOCI.dfso[SOCI.dfso['term'] == '4Φ']
display(df)
print('wt sum = {:.3f}'.format(df.wt.sum()))

In [ ]:
SOCI.dfso[(SOCI.dfso.exc > 21000) & (SOCI.dfso.exc < 30000)]